In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import gc
from fastai import datasets,layers
from fastai.vision import ImageList
from fastai.basic_train import Learner
from fastai.basic_data import DataBunch
from fastai.callback import Callback
from ipdb import set_trace
from torch.utils import data
from torch.utils.tensorboard import SummaryWriter
from functools import partial
import torchvision.datasets as tdatasets
import torch.nn as nn
import torch
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from time import time
import os
import psutil
from shutil import rmtree
import math
import random
import torch.nn.functional as F
import utils
import hdbscan
import umap.umap_ as umap

%load_ext line_profiler
utils.reload()

In [ ]:
NZ=50

In [ ]:
def reload():
    import importlib, utils
    importlib.reload(utils)
mnist_ds = utils.get_mnist_dset(x_only=True)
mnist_train = utils.get_mnist_dset(x_only=False)
type(mnist_ds[0])

In [ ]:
def entropy(t,dim):
#     x=t+torch.logsumexp(t,dim=0)
    x=t/t.std(dim=dim,keepdim=True)
    x = F.softmax(x,dim=dim)
    y = (x+1e-5).log()
    assert t.shape == x.shape and t.shape== y.shape
    ent = (-x*y).sum(dim=dim)
    return ent
a = torch.randn(3,6,8)
entropy(a,dim=0)

In [ ]:
enc,dec,lin = utils.get_enc_dec('cuda',latent_size=NZ)

In [ ]:
class Exemplar():
    def __init__(self,tensor): self.tensor,self.count,self.age=tensor,0,0
        
class Concepts():
    def __init__(self,dim,opt,thresh,num_starting_concepts=10): 
        self.dim,self.opt,self.thresh=dim,opt,thresh
        self.exemplars = []; params = []
        self.dist_func = nn.MSELoss()
        self.exemplars = [Exemplar(normalize(torch.randn(self.dim,device='cuda'))) for _ in range(num_starting_concepts)]
        params = [ex.tensor for ex in self.exemplars]

        assert len(self) == num_starting_concepts == len(params)
        self.opt.param_groups = self.opt.param_groups[:2]
        self.opt.add_param_group({'params': params, 'lr':1e-5})
        
    def add_exemplar(self,tensor):
        new_exemplar = Exemplar(tensor)
        self.exemplars.append(new_exemplar)
        return new_exemplar
 
    @property
    def exemplars_as_tensor(self): return torch.stack([e.tensor for e in self.exemplars])
    def __len__(self): return len(self.exemplars)
        
    def interpret_wo_grad(self,inp_,eps):
        for e in self.exemplars:
            e.count+=1
            e.age+=1
            if e.count - math.log(e.age) > 20: 
                self.exemplars.remove(e)
        try: dist,idx = torch.min((self.exemplars_as_tensor-inp_).norm(dim=-1),0)
        except RuntimeError: pass # No exemplars defined yet
        if len(self) == 0 or dist > self.thresh:
            best_match = self.add_exemplar(inp_.data.clone().squeeze(0))
            dist = torch.tensor(0.,device='cuda')
        else:
            best_match = self.exemplars[idx]
            best_match.count = 0
            best_match.tensor.lerp_(inp_.data.clone().squeeze(0), eps/(best_match.age+1))
        #best_match.tensor /= best_match.tensor.norm()
        #for e in self.exemplars: e.tensor.lerp_(best_match.tensor,-eps/(e.age+1))
#         if len(self) > 4 and self.thresh < 10: self.thresh *= 1.05
#         elif len(self) < 4: self.thresh *= 0.95
        return best_match, dist 
    
    
class ConceptLearner():
    def __init__(self,enc,dec,lin,dataset,opt,loss_func,discrim=None): 
        self.enc,self.dec,self.lin,self.dataset,self.loss_func,self.ce_loss_func = enc,dec,lin,dataset,loss_func,nn.CrossEntropyLoss()
        self.opt = opt(params = [{'params':enc.parameters()},{'params':dec.parameters()}])
        self.discrim = discrim if discrim else utils.NonsenseDiscriminator().to('cuda')
        self.discrim_opt = opt(params=[{'params': self.discrim.parameters()}])
        try: rmtree('runs')
        except: pass
        self.writer = SummaryWriter()
        self.init_concepts(1.)
        
    def init_concepts(self,thresh): self.concepts = Concepts(NZ,opt=self.opt,thresh=thresh,num_starting_concepts=0)
    def test_shapes(self): test_shapes(self.ae)
    def show_one_concept(self,idx=0): 
        plt.imshow(self.dec((self.concepts.exemplars[idx].tensor)[None,:,None,None]).squeeze())
        plt.show()
        
    def train_ae(self,epochs,bs,gan_lmbda,nonsense_lmbda,ohe_lmbda_max):
        dl = data.DataLoader(self.dataset,batch_sampler=data.BatchSampler(data.SequentialSampler(self.dataset),bs,drop_last=True),pin_memory=False)        
        for param in self.enc.parameters(): param.requires_grad = True
        for param in self.dec.parameters(): param.requires_grad = True    
        purities = [torch.tensor(0.) for i in range(4)]
        counts = [torch.tensor(0.) for i in range(4)]
        def purity_thresh(epoch,i): return 1-0.8*((epoch+i/len(dl))/epochs)
        thresh = 1.
        self.lin.weight.data -= self.lin.weight.mean(dim=-1)[:,None]
        self.lin.bias.data.zero_()
#         assert torch.allclose(self.lin.weight.mean(dim=-1),torch.zeros(NZ,device='cuda'),rtol=1e-2,atol=1e-3)
        for epoch in range(epochs):
            print(enc[1][3][0].weight.shape)
            ohe_lmbda=(epoch/epochs)*ohe_lmbda_max
            for i, xb in enumerate(dl): 
                latent = self.enc(xb)
                total_latents = latent.detach().cpu() if i==0 else torch.cat([total_latents,latent.detach().cpu()],dim=0)
#                 print(len(total_latents))
#                 latent = latent/latent.norm(dim=1,keepdim=True)
                preds = self.lin(latent[:,:,0,0])
#                 ohe_loss = Categorical(probs=latent[:,:,0,0]).entropy().mean()
                if ohe_lmbda_max > 0:
                    target_category, clatent = utils.oheify(preds)
                    ohe_loss = self.ce_loss_func(preds,target_category)
                else: 
                    clatent = latent
                    target_category = torch.argmax(preds,dim=1)
                    ohe_loss = torch.tensor(0.).cuda()
                rloss = self.loss_func(self.dec(latent),xb).mean(dim=[1,2,3])
                rpredloss = self.loss_func(self.dec(clatent),xb).mean(dim=[1,2,3])
                entropy_loss = entropy(preds,dim=1)
                easys = entropy_loss < 0.015
                if epoch>4:
                    if easys.sum() > 1: print(easys.sum().item())
                    centroid_dists = (centroids-latent[:,None,:,0,0]).norm(dim=-1)
                    first,second = torch.topk(cehntroid_dists,2,dim=-1)
                    set_trace()
                    
                loss = utils.safemean(rloss[~easys]) + utils.safemean((rpredloss+ohe_loss)[easys]) + 1*latent[:,:,0,0].norm(dim=-1).mean()+ohe_lmbda*ohe_loss
#                 try:purities = [torch.lerp(purity,torch.sum(rloss[target_category==i].detach().cpu()),0.1) for i,purity in enumerate(purities)]
#                 except: set_trace()
                counts = [torch.lerp(count,torch.sum(target_category.detach().cpu()==i).float(),0.5) for i,count in enumerate(counts)]
                loss.backward(retain_graph=True);
                self.opt.step(); self.opt.zero_grad(); self.discrim_opt.zero_grad() 
                self.writer.add_scalar('ae/loss',loss,i)
#             self.writer.add_images('Test_batch', pred,epoch)
            print(loss.mean().item(),entropy_loss,counts)
            if epoch>3 or True:
                umapped_latents = umap.UMAP(random_state=42).fit_transform(total_latents.squeeze())
                hdbscan_labels = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=500).fit_predict(umapped_latents)
                ohe_labels = torch.tensor(hdbscan_labels[:,None]) == torch.arange(max(hdbscan_labels)+1)
                d = utils.Dataholder(self.dataset.data,torch.tensor(hdbscan_labels,device='cuda'))
                self.labeled_ds = utils.TransformDataset(d,[utils.to_float_tensor,utils.add_colour_dimension],x_only=False,device='cuda')
                self.lin1 = nn.Linear(NZ,max(hdbscan_labels)).to('cuda')
                self.c_loss_func=nn.CrossEntropyLoss(reduction='none')
                self.train_epoch_labels(bs)
#                 centroids = torch.stack([total_latents[hdbscan_labels==i,:,0,0].mean(axis=0) for i in range(max(hdbscan_labels)+1)]).cuda()
            
            if epoch>2 and False:
                set_trace()
                to_remove = [count for count in counts if count<0.01]
                if len(to_remove) > 0: print('removing', to_remove)
                for item in to_remove: 
                    utils.prune(enc,dec,lin,counts.index(item))
                    del purities[counts.index(item)]
                    counts.remove(item)
                if len(to_remove) == 0: thresh*=0.95
                to_separate = [p for p in purities if p>thresh]
                if len(to_separate)>0: print('separating', to_separate)
                for item in to_separate:
                    idx = purities.index(item)
                    utils.duplicate_dim(enc,dec,lin,idx)
                    purities[idx] = torch.tensor(0.)
                    purities += [torch.tensor(0.)]
                    counts += counts[idx:idx+1]
                    
    def train_epoch_labels(self,bs):
        dl = data.DataLoader(self.labeled_ds,batch_sampler=data.BatchSampler(data.SequentialSampler(self.dataset),bs,drop_last=True),pin_memory=False)        
        for i, (xb,yb) in enumerate(dl):
            latent = self.enc(xb)
            c_pred = self.lin1(latent[:,:,0,0])
            r_pred = self.dec(latent)
            mask = yb>=0
            yb_ = yb*mask
            c_loss_ = self.c_loss_func(c_pred,yb_)
            print(c_loss_)
            c_loss = (c_loss_[mask]).mean()
            r_loss = self.loss_func(r_pred,xb)
            loss = r_loss.mean() + c_loss
            print(loss)
            loss.backward(); self.opt.step(); self.opt.zero_grad()
            
    
    def check_ae_images(self,num_rows=5):
        idxs = np.random.randint(0,len(self.dataset),size=num_rows*4)
        inimgs = self.dataset[idxs]
        x = self.enc(inimgs)
#         x = self.lin(x[:,:,0,0])[:,:,None,None]
        outimgs = self.dec(x)
        _, axes = plt.subplots(num_rows,4,figsize=(7,7))
        for i in range(num_rows):
            axes[i,0].imshow(inimgs[i,0])
            axes[i,1].imshow(outimgs[i,0])
            axes[i,2].imshow(inimgs[i+num_rows,0])
            axes[i,3].imshow(outimgs[i+num_rows,0])

    def check_concept_images(self,num_imgs=5):
        outimgs = self.dec(self.concepts.exemplars_as_tensor[:25,:,None,None]).squeeze(1)
        num_imgs = min(len(outimgs),25)
        _, axes = plt.subplots(5,5,figsize=(7,7))
        for i in range(num_imgs):
            axes.flatten()[i].imshow(outimgs[i])
    def vis_latent(self,l): plt.imshow(self.dec(l[None,:,None,None].cuda())[0,0]); plt.show()
    def vis_ae(self,x): vis_latent(self.enc(x.cuda()).squeeze())

In [ ]:
ones = mnist_train.x[(mnist_train.y == 1)].to('cuda')
twos = mnist_train.x[(mnist_train.y == 2)].to('cuda')
threes = mnist_train.x[(mnist_train.y == 3)].to('cuda')
fours = mnist_train.x[(mnist_train.y == 4)].to('cuda')
fives = mnist_train.x[(mnist_train.y == 5)].to('cuda')
sixes = mnist_train.x[(mnist_train.y == 6)].to('cuda')
sevens = mnist_train.x[(mnist_train.y == 7)].to('cuda')
eights = mnist_train.x[(mnist_train.y == 8)].to('cuda')
nines = mnist_train.x[(mnist_train.y == 9)].to('cuda')
simple_ds = utils.TransformDataset(torch.cat([ones,twos,fives,eights],dim=0), [utils.to_float_tensor,utils.add_colour_dimension],x_only=True,device='cuda')
simple_clearner = ConceptLearner(enc,dec,lin,simple_ds,opt=torch.optim.Adam,loss_func=nn.L1Loss(reduction='none'))
# %lprun -f simple_clearner.train_ae simple_clearner.train_ae(epochs=3,bs=64,gan_lmbda=0.01,nonsense_lmbda=1)
# simple_clearner.train_ae(epochs=25,bs=32,gan_lmbda=0.00,nonsense_lmbda=0.00,ohe_lmbda=0)
# simple_clearner.check_ae_images()
simple_clearner.train_ae(epochs=10,bs=64,gan_lmbda=0.01,nonsense_lmbda=0.0,ohe_lmbda_max=0.1)
simple_clearner.check_ae_images()

In [ ]:
for t in range(enc[1][3][0].weight.shape[0]):
    simple_clearner.vis_latent((torch.arange(enc[1][3][0].weight.shape[0])==t).float().cuda())

In [ ]:
reload()
def check(number):
    dl = utils.get_dloader(number,x_only=True)
    xb = next(iter(dl))
    return enc(xb).argmax(dim=1)[:20]
check(fives)

In [ ]:
simple_clearner.train_concepts(epochs=1,bs=1,gan_lmbda=0.01,eps=0.001,thresh=0.55)
simple_clearner.check_concept_images()
[e.age for e in simple_clearner.concepts.exemplars]

In [ ]:
l1 = enc(to_float_tensor(add_colour_dimension(twos[:2000])))
l1 /= l1.norm(dim=1,keepdim=True)
l1 = l1.mean(dim=0)[:,0,0]
l1.shape

In [ ]:
l2 = enc(to_float_tensor(add_colour_dimension(fours[:2000])))
l2 /= l2.norm(dim=1,keepdim=True)
l2=l2.mean(dim=0)
l3 = enc(to_float_tensor(add_colour_dimension(eights[:2000])))
l3 /= l3.norm(dim=1,keepdim=True)
l3=l3.mean(dim=0)
print(l1.shape,l2.shape,l3.shape)
simple_clearner.vis_latent(l1[:,0,0])
simple_clearner.vis_latent(l2[:,0,0])
simple_clearner.vis_latent(l3[:,0,0])

In [ ]:
def test_enc(): 
    plt.imshow(dec(utils.normalize(torch.randn(1,NZ,1,1).cuda())).squeeze())
    plt.show()
test_enc()

In [ ]:
def is_one_of(mod,mod_types):
    if mod_types == 'all': return True
    res = False
    for mod_type in mod_types: res = res or isinstance(mod,mod_type)
    return res

def printshapes(mod,inp,outp):
    print(f'\nInside {mod.__class__.__name__} forward')
    print(f'Input shape: {inp[0].shape}')
    print(f'Output shape: {outp.data.shape}')   
    
def printstats(mod,inp,outp):
    print(f'\nInside {mod.__class__.__name__} forward')
    print(f'Input mean: {inp[0].mean()}')
    print(inp[0][0])
    print(f'Output mean: {outp.data.mean()}')   
    print(outp[0][0])
    
def printshapesstats(mod,inp,outp):
    printshapes(mod,inp,outp)
    printstats(mod,inp,outp)
    
def hook_layers_of_types(model,hook,layertypes):
    hooks = []
    for child in model.children():
        if is_one_of(child,layertypes): hooks.append(child.register_forward_hook(hook))
        hooks += hook_layers_of_types(child,hook,layertypes)
    return hooks

def test_model(model,test_batch,test_func,layertypes):
    added_hooks = hook_layers_of_types(model,test_func,layertypes)
    try: model(test_batch)
    except Exception as e: print(e)
    finally:
        for hook in added_hooks: hook.remove()

In [ ]:
# enc,dec = utils.get_enc_dec(device='cuda', latent_size=NZ)
# enc,dec,mnist_dataset.data = enc.to('cuda'),dec.to('cuda'),mnist_dataset.data.to('cuda')
dl = utils.get_dloader(ones,x_only=True)
test_batch = next(iter(dl))
test_model(enc,test_batch,printshapes,'all')